In [ ]:
#Run any one cell to select a certain url

In [92]:
#Alphabetical parse url
url = "https://www.espncricinfo.com/ci/content/player/country.html?country=6;alpha=A"

In [109]:
#Country based parse url
url = "https://www.espncricinfo.com/ci/content/player/index.html?country=6"

In [110]:
from bs4 import BeautifulSoup as bf
import requests
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [111]:
req = requests.get(url)

In [112]:
soup = bf(req.text,"html.parser")

In [113]:
search_tag_container = soup.find_all('td')

In [114]:
#Fetch total urls of players belonging to a country
u = list()
for job_elem in search_tag_container:
    u.append(job_elem.find('a'))
    #result.append(job_elem.text)
l = set(u)
l = list(l)

In [115]:
def evaluate(html):
  select_divs = html.find_all('div',class_="card overflow-hidden mb-3")
  exist = 0
  location = 0
  for i in range(len(select_divs)):
    check = ""
    try:
      check = select_divs[i].find('p',class_="benton-bold pl-3 pt-4 pb-3 m-0 player-card-header").text
    except:
      k = 1
    if(check == "Career Averages"):
      exist = 1
      location = i
      break
  return exist,select_divs[location]

In [116]:
#Get relevant information related to a player using each player's url
def load(req,final_list_players,ind,u,debut_url):
    lab = list()
    result = list()
    soup = bf(req.text,"html.parser")
    print(soup.title)
    search_tag_container = soup.find_all('h5',class_='player-card-description gray-900')
    count = 0
    for job_elem in search_tag_container:
        result.append(job_elem.text)
        count+=1
    search_tag_container = soup.find_all('p',class_="text-uppercase gray-700 mb-0 pb-0-5 player-card-heading")
    for job_elem in search_tag_container:
        lab.append(job_elem.text)
    ######################################
    rec = extra(u)
    lab.append("Records")
    result.append([rec])
    print(result)
    ##########################################
    matches = debut(debut_url)
    result.append([matches])
    lab.append("Matches")
    ##########################################
    lab.append("Teams")
    team_data = teams_data(soup)
    result.append([team_data])
    ###########################################
    search_tag_container = soup.find('div',class_="player-card__details")
    l = list()
    for job_elem in search_tag_container:
        l.append(job_elem.text)
        count+=1
    p = l[1].split("|")
    l.pop()
    l.pop()
    l = l+p
    result = result + l
    l_label = ["Player Name" , "Nationality ", "player-type"]
    lab = lab + l_label
    ##########################
    valid,soup = evaluate(soup)
    if(valid == 1):
      search_tag_container = soup.findAll('thead')
      if(len(search_tag_container)!=0):
        tag_content = search_tag_container[0].find_all('th')
        head = 0
        table_labels = list()
        table_labels_1 =list()
        for job_elem in tag_content:
            table_labels.append(job_elem.text)
            head+=1
        print('Head count',head)
        
        search_tag_container = soup.findAll('tbody')
        tag_content = search_tag_container[0].find_all('td')
        track_head_label = 0
        for job_elem in tag_content:
            if(track_head_label%head !=0):
                result.append(job_elem.text)
                count+=1
            else:
                table_labels_1.append(job_elem.text)
            
            track_head_label+=1
        total_label1 = list()
        for i in range(1,len(table_labels)):
            for j in range(len(table_labels_1)):
                total_label1.append('Batting' + table_labels[i] +" "+table_labels_1[j])
        lab = lab + total_label1
      ####################################
      search_tag_container = soup.findAll('thead')
      if(len(search_tag_container)>1):
        tag_content = search_tag_container[1].find_all('th')
        head = 0
        table_labels = list()
        table_labels_1 =list()
        for job_elem in tag_content:
            table_labels.append(job_elem.text)
            head+=1
        print('Head count',head)
        
        search_tag_container = soup.findAll('tbody')
        tag_content = search_tag_container[1].find_all('td')
        track_head_label = 0
        for job_elem in tag_content:
            if(track_head_label % head !=0):
                result.append(job_elem.text)
                count+=1
            else:
                table_labels_1.append(job_elem.text)
            
            track_head_label+=1
        total_label1 = list()
        for i in range(1,len(table_labels)):
            for j in range(len(table_labels_1)):
                total_label1.append('Bowling' + table_labels[i] +" "+table_labels_1[j])
        lab = lab + total_label1
    print('label--',len(lab))
    print('result--',len(result))
    
    lo = dict(zip(lab,result))
    lp = pd.DataFrame(lo,index=[ind])
    final_list_players.append(lp)
    print('count',count)

In [117]:
#Get individual player's "Records" information.
import re
def extra(l):
    re = requests.get(l)
    time.sleep(2)
    so = bf(re.text,"html.parser")
    res1 = so.find_all('div',class_='col-14 d-flex flex-row')
    res = so.find_all('div',class_='row bg-gray-100 class-record-row-p d-flex align-items-center')
    #print(res1)
    url_list = list()
    for i in range(len(res1)):

        pre = res1[i].find_all('h1')
        loi = res1[i].find_all('p',class_='record-class-title m-0')
        #print(loi)    
        for j in range(len(pre)):
            #print(pre[j].text)
            #print(loi[j].text)
            url_list.append(str(pre[j].text)  +str(" ") + str(loi[j].text))
    return url_list


In [118]:
def teams_data(so):
  teams = []
  loi = so.find_all('h5',class_='m-0 ml-2 link-border-bottom player-description-link')
  for i in range(len(loi)):
    teams.append(loi[i].text)
  return teams

In [119]:
def debut(link):
  player_match = list()
  url = link
  request = requests.get(url)
  soup = bf(request.text,"html.parser")
  search_tag_container = soup.find_all('div',{'id':'debut-last-matches'})
  cp_jo = search_tag_container
  if(len(search_tag_container) != 0):
    total = cp_jo[0].find_all('div',class_="")
    for i in range(len(total)):
      p =total[i]
      a= p.find('h5',{'class':"benton-bold player-matches-subtitle d-flex justify-content-between align-items-center player_matches-dropdown m-0"})
      k = total[i].find_all('span',class_="match-event-row-debut-last white-space-nowrap")
      search_tag_container = total[i].find_all('span',{"class": "player-match-link"})
      new = total[i].find_all('div',{"class":"col-12 player-match-event-cell white-space-nowrap"})
      #print(a)
      #print(k[0])
      #print(new)
      try:
        l =a.span.decompose()
      except:
        player_match.append(a.text)
        #print(a.text)
      else:
        player_match.append(a.text)
        #print(a.text)
      for j in range(len(k)):
        try:
          temp = search_tag_container[j].text
        except:
          temp = new[j].text
        append = k[j].text + str(" ") + temp
        player_match.append(append)
        #print(k[j].text,end=" ")
        #print(search_tag_container[j].text)
  return player_match

In [120]:
import time
final_list_players = list()
for i in range(3):
    u = "https://www.espncricinfo.com"
    rl = l[i]["href"]
    link = u+rl
    print(link)
    req = requests.get(link)
    time.sleep(2)
    print('player-',i,end=" ")
    ind = i-1
    record_url = req.url+str("/tests-odi-t20-records")
    debut_url = req.url+str("/matches")
    #debut(debut_url)
    load(req,final_list_players,ind,record_url,debut_url)
    #print(record_url)
    #jk = extra(record_url)
    #print(jk)

https://www.espncricinfo.com/ci/content/player/931581.html
player- 0 <title>Rishabh Pant profile and biography, stats, records, averages, photos and videos</title>
['Rishabh Rajendra Pant', 'October 04, 1997, Haridwar, Uttarakhand', '23y 234d', 'Left hand bat', 'Wicketkeeper', 'Wicketkeeper batter', [['1st Most dismissals in a match (11)', '1st Most catches in a match (11)', ' 300 runs and 15 wicketkeeping dismissals in a series ']]]
Head count 15
Head count 14
label-- 174
result-- 174
count 171
https://www.espncricinfo.com/ci/content/player/326016.html
player- 1 <title>Bhuvneshwar Kumar profile and biography, stats, records, averages, photos and videos</title>
['Bhuvneshwar Kumar Singh', 'February 05, 1990, Meerut, Uttar Pradesh', '31y 110d', 'Right hand bat', 'Right arm medium', 'Bowler', [['8th Outstanding bowling analyses in an innings (4/8)', '4th Most runs conceded in an innings (106)', '1st Most maidens in an innings (2)']]]
Head count 14
Head count 15
label-- 174
result-- 174
c

In [121]:
a = final_list_players[0]

In [122]:
len(final_list_players)

3

In [123]:
df = a.T
for i in range(1,len(final_list_players)):
    lo = final_list_players[i].T
    df = pd.concat([df,lo],axis=1)

In [124]:
df.T.shape

(3, 277)

In [125]:
df.T

,Full Name,Born,Age,Batting Style,Fielding Position,Playing Role,Records,Matches,Teams,Player Name,Nationality,player-type,BattingMat Test,BattingMat ODI,BattingMat T20I,BattingMat FC,BattingMat List A,BattingMat T20,BattingInns Test,BattingInns ODI,BattingInns T20I,BattingInns FC,BattingInns List A,BattingInns T20,BattingNO Test,BattingNO ODI,BattingNO T20I,BattingNO FC,BattingNO List A,BattingNO T20,BattingRuns Test,BattingRuns ODI,BattingRuns T20I,BattingRuns FC,BattingRuns List A,BattingRuns T20,BattingHS Test,BattingHS ODI,BattingHS T20I,BattingHS FC,BattingHS List A,BattingHS T20,BattingAve Test,BattingAve ODI,BattingAve T20I,BattingAve FC,BattingAve List A,BattingAve T20,BattingBF Test,BattingBF ODI,BattingBF T20I,BattingBF FC,BattingBF List A,BattingBF T20,BattingSR Test,BattingSR ODI,BattingSR T20I,BattingSR FC,BattingSR List A,BattingSR T20,Batting100s Test,Batting100s ODI,Batting100s T20I,Batting100s FC,Batting100s List A,Batting100s T20,Batting50s Test,Batting50s ODI,Batting50s T20I,Batting50s FC,Batting50s List A,Batting50s T20,Batting4s Test,Batting4s ODI,Batting4s T20I,Batting4s FC,Batting4s List A,Batting4s T20,Batting6s Test,Batting6s ODI,Batting6s T20I,Batting6s FC,Batting6s List A,Batting6s T20,BattingCt Test,BattingCt ODI,BattingCt T20I,BattingCt FC,BattingCt List A,BattingCt T20,BattingSt Test,BattingSt ODI,BattingSt T20I,BattingSt FC,BattingSt List A,BattingSt T20,BowlingMat Test,BowlingMat ODI,BowlingMat T20I,BowlingMat FC,BowlingMat List A,BowlingMat T20,BowlingInns Test,BowlingInns ODI,BowlingInns T20I,BowlingInns FC,BowlingInns List A,BowlingInns T20,BowlingBalls Test,BowlingBalls ODI,BowlingBalls T20I,BowlingBalls FC,BowlingBalls List A,BowlingBalls T20,BowlingRuns Test,BowlingRuns ODI,BowlingRuns T20I,BowlingRuns FC,BowlingRuns List A,BowlingRuns T20,BowlingWkts Test,BowlingWkts ODI,BowlingWkts T20I,BowlingWkts FC,BowlingWkts List A,BowlingWkts T20,BowlingBBI Test,BowlingBBI ODI,BowlingBBI T20I,BowlingBBI FC,BowlingBBI List A,BowlingBBI T20,BowlingBBM Test,BowlingBBM ODI,BowlingBBM T20I,BowlingBBM FC,BowlingBBM List A,BowlingBBM T20,BowlingAve Test,BowlingAve ODI,BowlingAve T20I,BowlingAve FC,BowlingAve List A,BowlingAve T20,BowlingEcon Test,BowlingEcon ODI,BowlingEcon T20I,BowlingEcon FC,BowlingEcon List A,BowlingEcon T20,BowlingSR Test,BowlingSR ODI,BowlingSR T20I,BowlingSR FC,BowlingSR List A,BowlingSR T20,Bowling4w Test,Bowling4w ODI,Bowling4w T20I,Bowling4w FC,Bowling4w List A,Bowling4w T20,Bowling5w Test,Bowling5w ODI,Bowling5w T20I,Bowling5w FC,Bowling5w List A,Bowling5w T20,Bowling10w Test,Bowling10w ODI,Bowling10w T20I,Bowling10w FC,Bowling10w List A,Bowling10w T20,Bowling Style,BattingBalls Test,BattingBalls ODI,BattingBalls T20I,BattingBalls FC,BattingBalls List A,BattingBalls T20,BattingWkts Test,BattingWkts ODI,BattingWkts T20I,BattingWkts FC,BattingWkts List A,BattingWkts T20,BattingBBI Test,BattingBBI ODI,BattingBBI T20I,BattingBBI FC,BattingBBI List A,BattingBBI T20,BattingBBM Test,BattingBBM ODI,BattingBBM T20I,BattingBBM FC,BattingBBM List A,BattingBBM T20,BattingEcon Test,BattingEcon ODI,BattingEcon T20I,BattingEcon FC,BattingEcon List A,BattingEcon T20,Batting4w Test,Batting4w ODI,Batting4w T20I,Batting4w FC,Batting4w List A,Batting4w T20,Batting5w Test,Batting5w ODI,Batting5w T20I,Batting5w FC,Batting5w List A,Batting5w T20,Batting10w Test,Batting10w ODI,Batting10w T20I,Batting10w FC,Batting10w List A,Batting10w T20,BowlingNO Test,BowlingNO ODI,BowlingNO T20I,BowlingNO FC,BowlingNO List A,BowlingNO T20,BowlingHS Test,BowlingHS ODI,BowlingHS T20I,BowlingHS FC,BowlingHS List A,BowlingHS T20,BowlingBF Test,BowlingBF ODI,BowlingBF T20I,BowlingBF FC,BowlingBF List A,BowlingBF T20,Bowling100s Test,Bowling100s ODI,Bowling100s T20I,Bowling100s FC,Bowling100s List A,Bowling100s T20,Bowling50s Test,Bowling50s ODI,Bowling50s T20I,Bowling50s FC,Bowling50s List A,Bowling50s T20,Bowling4s Test,Bowling4s ODI,Bowling4s T20I,Bowling4s FC,Bowling4s List A,Bowling4s T20,Bowling6s Test,Bowling6s OD